## Kryptografia w bezpieczeństwie sieci bezprzewodowych
#### Adrianna Kralka
#### Jakub Młocek
#### Marcel Trzaskawka

### Spis treści
- Teoria	
    - Kryptografia w sieciach	
    - WEP Wired Equivalent Privacy	
        - Opis:	
        - Szyfr RC4:	
        - Suma Kontrolna CRC-32:	
        - Wykryte podatności:	
        - Wnioski:	
- WPS Wifi protected setup	
- Bluetooth	
- WPA
- Standard przejściowy między WEP, a WPA2. EAP, Temporal Key Integruty      Protocol (TKIP), RC4,i Messege Integrity Check (Michael algorithm)	
- WPA2	
    - CCMP
    - Wykryta podatność:	
    - Praktyczne ukazanie ataku KRACK na WPA2:	
    - Wnioski:	
- WPA3
- CCMP/AES (WPA3-Personal)
- GCM/AES (WPA3-Enterprise
- WPA-PSK zastąpiony Simultaneous Authentication of Equals (SAE)	
- Źródła


## WEP Wired Equivalent Privacy
- Opis:
Standard Wired Equivalent Privacy (WEP) został wprowadzony w 1999 roku jako część oryginalnego standardu 802.11. Miało to na celu zapewnienie poufności na poziomie zbliżonym do sieci kablowych. W standardzie używany jest szyfr strumieniowy RC4 który miał zapewniać zachowanie poufności poufności oraz suma kontrolna CRC-32 dla zapewnienia integralności. Szyfrowanie występuje w dwóch wersjach z różną długością klucza RC4: 64 i 128 bitów. <br>
<br>
- Szyfr RC4:
RC4 jest szyfrem strumieniowym. Szyfrowanie za pomocą RC4 polega na sumowaniu XOR kolejnych bajtów wiadomości z kolejnymi bajtami strumienia klucza. Szczegóły działania szyfru:

1. Inicjowanie tabeli T - Tabela T ma długość 256 bajtów. Jest wypełniana bajtami na podstawie wartości sekretnego klucza, przed rozpoczęciem operacji szyfrowania lub deszyfrowania. Następujące operacje powinny zostać wykonane, aby utworzyć tabelę:
- W każdą komórkę tablicy wpisywana jest liczba odpowiadająca numerowi jej pozycji (licząc od zera).
- Tworzona jest zmienna pomocnicza i inicjowana na 0.
- Dla każdej liczby w tablicy (numery przyjmują wartości kolejno od 0 do 255) wykonywane są dwie operacje: <br>
     <br>   1.1. Wartość zmiennej pomocniczej jest modyfikowana zgodnie ze wzorem:
-              X = (X + W + K) mod 256, gdzie:
     - X - zmienna pomocnicza,
     - W - wartość aktualnej pozycji w tabeli T,
     - K - wartość bajtu klucza w aktualnej pozycji podzielonej modulo przez długość klucza (ponieważ sekretny klucz może być krótszy niż 256 bajtów),

     1.2.Liczba w tablicy w aktualnej pozycji jest zamieniana miejscem z liczbą w tabeli w pozycji określonej przez zmienną pomocniczą.

2. Szyfrowanie i Deszyfrowanie - W trakcie szyfrowania i deszyfrowania generowane są bajty strumienia klucza, które następnie sumuje się XOR z bajtami wiadomości Bajty strumienia klucza produkowane są na podstawie tablicy T. Następujące kroki są wykonywane:

- Tworzone są dwie zmienne pomocnicze p1 i p2, obie inicjowane są na 0.
- Do zmiennej p1 dodaje się 1, a wynik dzieli modulo przez 256.
- Do zmiennej p2 dodaje się wartość tablicy na pozycji p1 (czyli T[p1]), a wynik dzieli modulo przez 256.
- Zamienia się miejscami wartości tablicy na pozycjach p1 i p2.
- Sumuje się dwie wartości tablicy na pozycjach p1 i p2, a wynik dzieli modulo przez 256. Następnie wynik przypisuje się do nowej zmiennej pomocniczej p3.
- Wartość tablicy na pozycji p3 jest kolejnym bajtem strumienia klucza.
- Jeśli potrzebne są kolejne bajty strumienia klucza, to należy wrócić do punktu II i powtórzyć powyższe kroki.
<br>
<br> Implementacja szyfru rc4 w języku python  na podstawie powyższych kroków:



In [ ]:
import codec
SIZE = 256
def createKeyTable(key):
   T = list(range(SIZE))
   pom = 0
   for i in range(SIZE):
       pom = (pom + T[i] + key[i % len(key)]) % SIZE
       T[i], T[pom] = T[pom], T[i]
   return T

def keyStreamGenerator(T):
   p1 = 0
   p2 = 0
   while True:
       p1 = (p1 + 1) % SIZE
       p2 = (p2 + T[p1]) % SIZE
       T[p1], T[p2] = T[p2], T[p1]
       p3 = (T[p1] + T[p2]) % SIZE
       yield p3

def getKeyStream(key):
   T = createKeyTable(key)
   return keyStreamGenerator(T)

def encrypt(key, plaintext):
   key = [ord(c) for c in key]
   if type(plaintext) == str:
       plaintext = [ord(c) for c in plaintext]
   keystream = getKeyStream(key)
   res = []
   for c in plaintext:
       val = ("%02X" % (c ^ next(keystream))) 
       res.append(val)
   return ''.join(res)


def decrypt(key, ciphertext):
   ciphertext = codecs.decode(ciphertext, 'hex_codec')
   res = encrypt(key, ciphertext)
   return codecs.decode(res, 'hex_codec').decode('utf-8')


if __name__ == "__main__":
   key = "zajprojektowezkryptografii"
   plaintext = "Mozeudanamsiedostac5zprojketu"
   ciphertext = encrypt(key,plaintext)
   print(f"plaintex: {plaintext}\nciphertext: {ciphertext}\n key: {key}")
   ciphertext = "8AEE24852B79ED2B6BC6FD005EE43344432D4889335E4DF55E4CB0F369"
   decryptedPlaintext = decrypt(key,ciphertext)
   print(f"decrytped plain text: {decryptedPlaintext}")

### Suma Kontrolna CRC-32:
Algorytm CRC (Cyclic Redundancny Check) 32bit służy do wyznaczania sum kontrolnych dla dowolnych danych wejściowych. Jest to nic innego jak 32-bitowa liczba całkowita określająca poprawność danych ze wzorcem. 
Podstawą działania algorytmu CRC-32 jest wielomian:
x^32+x^26+x^23+x^22+x^16+x^12+x^11+x^10+x^8+x^7+x^5+x^4+x^2+x^1+x^0
Wielomian ten jest reprezentowany jako pojedyncza liczba 32-bitowa. Każdy fragment
to bit o wartości 1 ustawiony w miejscu wyznaczonym przez wykładnik potęgi:
100000100110000010001100110110111 (2)
Jej długość wynosi 33 bity więc o jeden bit za dużo. Usuwamy najstarszy bit i otrzymujemy 32-bitową reprezentację wielomianu CRC-32.
00000100110000010001100110110111 (2) = 04C11DB7 (16) 
Wartość ta jest zawsze stała. <br>
<br> 
Następnie dla kazdego kodu ASCII obliczamy wartości:



In [ ]:
for(ascii = 0; ascii < 255; ascii = ascii + 1)   
   A[ascii] = ascii;
   odwróć 8 najmłodszych bitów A[ascii];
   A[ascii] = A[ascii] << 24;                       # przesuń o 24 bity w lewo
   for(i = 0; i < 8; i = i + 1)                     # petla od 0 do 7
     if((A[ascii] AND 0x80000000) != 0)             # jesli jest rożne od 0
       A[ascii] = (A[ascii] << 1) XOR 0x04C11DB7;   # przesuń o 1 bit w lewo i xor'uj z 0x04C11DB7
     else
       A[ascii] = A[ascii] << 1;                    # przesuń o 1 bit w lewo
   odwróć 32 bity A[ascii];

   #Wartości wyliczone w powyższym algorytmie są zazwyczaj obliczane raz 
   # i podawane jako stała. 
   # Teraz możemy już obliczyć sumę kontrolną dla dowolnego ciągu danych:

crc32 = 0xFFFFFFFF;   // wartość początkowa
for(i = 0; i < n; i = i + 1)   // pętla od 0 do n - 1
   crc32 = (crc32 >> 8) XOR A[(crc32 AND 0xFF) XOR B[i]];
crc32 = crc32 XOR 0xFFFFFFFF;